In [2]:
from ROOT import *
import numpy as np
from prettytable import PrettyTable
import sys
import uncertainties as unc # propagate uncertainties
from uncertainties import unumpy as unp # array operations for type ufloat
import os
    
var = "bdt"  #change the var name according to the var you want to read
mc = "pythia-eta-test"   #by setting it as "SF" or "MC", it will automatically making scale factor plots or MC closure plots

if var == "ntrk":
    doreweight = "Quark"  
if var == "bdt":
    doreweight = "Quark" 
    
eta_bin = [0,0.4,0.8,1.4]

reweight_method_array =[0,1,2,3]




fdijetMC = TFile("/eos/user/r/rqian/dijet-mono-result/pythia-eta-test/dijet_pythia_etatest_1-4.root")

fdijetData = TFile("/eos/user/r/rqian/dijet-mono-result/data-eta-test/dijet_data_1-4.root")
bins = np.array([500.,600.,800.,1000.,1200.,1500.,2000.])
bin = np.array([500,600,800,1000,1200,1500,2000])

path = mc+"_distribution"
isExists=os.path.exists(path)
if not isExists:
        os.makedirs(path)        

var_path = "plots_" + var
isExists=os.path.exists(var_path)
if not isExists:
        os.makedirs(var_path)        

        
#convert histogram and error into unp.uarray
#if sample is pyroot input, GetBinError returns correct result
def unc_array(hist):
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = hist.GetBinError(j)
    result = unp.uarray(value,error)
    return(result)

#convert histogram and error into unp.uarray
#if sample is uproot input, err is the sumw2 of the corresponding histogram
def unc_array_err(hist,err):
    
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = np.sqrt(err.GetBinContent(j))
    result = unp.uarray(value,error)
    return(result)


def myText(x,y,text, color = 1):
    l = TLatex()
    l.SetTextSize(0.025)
    l.SetNDC()
    l.SetTextColor(color)
    l.DrawLatex(x,y,text)
    pass

def error_check(hist):
    for i in range(1,hist.GetNbinsX()+1):
        if hist.GetBinContent(i)>1:
            hist.SetBinContent(i,0)
    return(hist)



# convert uncertainty array into histogram
def set_hist(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))
        hist.SetBinContent(i,unp.nominal_values(unc[i-1]))
        
def set_hist_error(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))

#inclusive fraction calculation
def fraction(lower_quark,lower_gluon,higher_quark,higher_gluon):
    ToT_Fq2 = 0.
    ToT_Fg2 = 0.
    ToT_Cq2 =0.
    ToT_Cg2 = 0.

    for j in range(1,lower_quark.GetNbinsX()+1):
        ToT_Fq2+=higher_quark.GetBinContent(j)
        ToT_Cq2+=lower_quark.GetBinContent(j)
        ToT_Fg2+=higher_gluon.GetBinContent(j)
        ToT_Cg2+=lower_gluon.GetBinContent(j)

    # calculate the fraction of each sample 
    if ((ToT_Fg2+ToT_Fq2) != 0):
        fg=ToT_Fg2/(ToT_Fg2+ToT_Fq2)
        cg=ToT_Cg2/(ToT_Cq2+ToT_Cg2)

    fq=1.-fg
    cq=1.-cg
    return(fg,cg,fq,cq)

def mc_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,fg,cg,fq,cq,higher_mc_input,lower_mc_input):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower_mc = lower_mc_input.Clone()
        higher_mc = higher_mc_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_quark)


            
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_gluon)

        
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower_mc.Integral() != 0):
                lower_mc.Scale(1./lower_mc.Integral())
        if(higher_mc.Integral() != 0):
                higher_mc.Scale(1./higher_mc.Integral())

        higher = higher_mc.Clone()
        lower = lower_mc.Clone()          


        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)

#return the abs systematic uncertainty
def mc_error(mc_quark_1,mc_gluon_1,mc_quark_2,mc_gluon_2): 
    error_q = np.zeros(mc_quark_1.GetNbinsX())
    error_g = np.zeros(mc_gluon_1.GetNbinsX())

    for j in range(1,mc_gluon_1.GetNbinsX()+1):
                q = abs(mc_quark_1.GetBinContent(j) - mc_quark_2.GetBinContent(j))
                g = abs(mc_gluon_1.GetBinContent(j) - mc_gluon_2.GetBinContent(j))
                error_q[j-1] = q
                error_g[j-1] = g

    return error_q,error_g

# do matrix on data for given normalized distribution
def data_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,higher_input,lower_input,fg,cg,fq,cq):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower = lower_input.Clone()
        higher = higher_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_quark)    
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_gluon)                                
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower.Integral() != 0):
            lower.Scale(1./lower.Integral()) 
        if(higher.Integral() != 0):
            higher.Scale(1./higher.Integral())         
        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()
        #if min == 500:
        #    for j in range(1,61):
        #        print("testflag3:",j,higher.GetBinContent(j),lower.GetBinContent(j))

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)
    
    
# Get the bin(cut) of given working point for the nominal result and the Quark efficiency , Gluon rejection , scale factor.
def wp_sf_nominal(wpoint,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])
    
    mc_bin = np.abs(quark_mc_cumsum -  wpoint*quark_mc.Integral(1,quark_mc.GetNbinsX()+1)).argmin()
    data_bin = np.abs(quark_data_cumsum -  wpoint*quark_data.Integral(1,quark_data.GetNbinsX()+1)).argmin()
    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(mc_bin,data_bin,sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

# Get the Quark efficiency , Gluon rejection , scale factor for systematics.
def wp_sf(mc_bin,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])

    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

#def bootstrap_result(data):
#    w = 58.45/39.91 # prescale factor 
#    n1 = data.Clone()
#    n2 = data.Clone()
#    result = data.Clone()
#    for j in range(1,higher_quark.GetNbinsX()+1):
#            n2.SetBinContent(j,int((data.GetBinContent(j)-data.GetBinError(j)**2)/(w-w**2)))#(events num for prescaled data)
#            n1.SetBinContent(j,int((data.GetBinContent(j)-w*n2.GetBinContent(j))))
#            result.SetBinContent(j,w*np.random.poisson(n2.GetBinContent(j))+np.random.poisson(n1.GetBinContent(j)))
#    return(result)  

def bootstrap_result(data):
    n1 = data.Clone()
    result = data.Clone()
    for j in range(1,higher_quark.GetNbinsX()+1):
            result.SetBinContent(j,np.random.poisson(n1.GetBinContent(j)))
    return(result)  

    
def ReadingDijetDataEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_data = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var)
    lower_data = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var)

    higher_data.SetName(pt+"_LeadingJet_Forward_Data_"+str(index)+var+name)
    lower_data.SetName(pt+"_LeadingJet_Central_Data_"+str(index)+var+name)
    
    higher_data_2 = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var)
    lower_data_2 = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var)
    
    higher_data_2.SetName(pt+"_SubJet_Forward_Data_"+str(index)+var+name)
    lower_data_2.SetName(pt+"_SubJet_Central_Data_"+str(index)+var+name)
    
    higher_data.Add(higher_data_2)
    lower_data.Add(lower_data_2)
    

    higher_data_err = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_err = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_err.SetName(pt+"_LeadingJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_err.SetName(pt+"_LeadingJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_2_err = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_2_err = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_2_err.SetName(pt+"_SubJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_2_err.SetName(pt+"_SubJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_err.Add(higher_data_2_err)
    lower_data_err.Add(lower_data_2_err)
    
    for i in range(1,higher_data.GetNbinsX()+1):
        higher_data.SetBinError(i,np.sqrt(higher_data_err.GetBinContent(i)))
        lower_data.SetBinError(i,np.sqrt(lower_data_err.GetBinContent(i)))
        
    histo_dic["hd"] =  higher_data
    histo_dic["ld"] =  lower_data

    return(histo_dic)        


def ReadingDijetMCEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_quark = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var)


    higher_quark.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+var+name)
    lower_quark.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+var+name)
    higher_gluon.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+var+name)
    

    higher_quark_2 = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark_2 = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon_2 = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon_2 = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark_2 = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark_2 = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark_2 = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark_2 = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var)

    higher_quark_2.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+var+name)
    lower_quark_2.SetName(pt+"_SubJet_Central_Quark_"+str(index)+var+name)
    higher_gluon_2.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon_2.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark_2.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark_2.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark_2.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark_2.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+var+name)

    higher_quark.Add(higher_quark_2)
    higher_gluon.Add(higher_gluon_2)
    higher_b_quark.Add(higher_b_quark_2)
    higher_c_quark.Add(higher_c_quark_2)

    lower_quark.Add(lower_quark_2)
    lower_gluon.Add(lower_gluon_2)
    lower_b_quark.Add(lower_b_quark_2)
    lower_c_quark.Add(lower_c_quark_2)
 
            
    higher_b_quark.Add(higher_c_quark)
    lower_b_quark.Add(lower_c_quark)

    higher_quark_err = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_err = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_err = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_err = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_err = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_err = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_err = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_err = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err")



    
    higher_quark_err.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_err.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_err.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_err.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_err.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_err.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_err.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_err.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_2_err = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_2_err = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_2_err = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_2_err = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_2_err = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_2_err = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_2_err = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_2_err = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err")
    
    higher_quark_2_err.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_2_err.SetName(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_2_err.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_2_err.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_2_err.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_2_err.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_2_err.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_2_err.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_err.Add(higher_quark_2_err)
    higher_gluon_err.Add(higher_gluon_2_err)
    higher_b_quark_err.Add(higher_b_quark_2_err)
    higher_c_quark_err.Add(higher_c_quark_2_err)

    lower_quark_err.Add(lower_quark_2_err)
    lower_gluon_err.Add(lower_gluon_2_err)
    lower_b_quark_err.Add(lower_b_quark_2_err)
    lower_c_quark_err.Add(lower_c_quark_2_err)
 
            
    higher_b_quark_err.Add(higher_c_quark_err)
    lower_b_quark_err.Add(lower_c_quark_err)
    
    for i in range(1,higher_quark.GetNbinsX()+1):
        higher_quark.SetBinError(i,np.sqrt(higher_quark_err.GetBinContent(i)))
        higher_gluon.SetBinError(i,np.sqrt(higher_gluon_err.GetBinContent(i)))
        higher_b_quark.SetBinError(i,np.sqrt(higher_b_quark_err.GetBinContent(i)))
        lower_quark.SetBinError(i,np.sqrt(lower_quark_err.GetBinContent(i)))
        lower_gluon.SetBinError(i,np.sqrt(lower_gluon_err.GetBinContent(i)))
        lower_b_quark.SetBinError(i,np.sqrt(lower_b_quark_err.GetBinContent(i)))

    
    histo_dic["hq"] =  higher_quark
    histo_dic["hg"] =  higher_gluon
    histo_dic["ho"] =  higher_b_quark
    
    histo_dic["hmc"] =  higher_quark.Clone()
    histo_dic["hmc"].SetName(pt+"higher_all_jets"+var+"_"+name)
    histo_dic["hmc"].Add(higher_gluon)
    histo_dic["hmc"].Add(higher_b_quark)
    
    histo_dic["lq"] =  lower_quark
    histo_dic["lg"] =  lower_gluon
    histo_dic["lo"] =  lower_b_quark
    
    histo_dic["lmc"] =  lower_quark.Clone()
    histo_dic["lmc"].SetName(pt+"lower_all_jets"+var+"_"+name)
    histo_dic["lmc"].Add(lower_gluon)
    histo_dic["lmc"].Add(lower_b_quark)
    
    return(histo_dic)

# Get the bin(cut) of given working point for the nominal result and the Quark efficiency , Gluon rejection , scale factor.
def wp_sf_nominal(wpoint,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])
    
    mc_bin = np.abs(quark_mc_cumsum -  wpoint*quark_mc.Integral(1,quark_mc.GetNbinsX()+1)).argmin()
    data_bin = np.abs(quark_data_cumsum -  wpoint*quark_data.Integral(1,quark_data.GetNbinsX()+1)).argmin()
    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(mc_bin,data_bin,sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

# Get the Quark efficiency , Gluon rejection , scale factor for systematics.
def wp_sf(mc_bin,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])

    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)


def bootstrap_result(data):
    n1 = data.Clone()
    result = data.Clone()
    for j in range(1,higher_quark.GetNbinsX()+1):
            result.SetBinContent(j,np.random.poisson(n1.GetBinContent(j)))
    return(result)  




for eta_reweight in reweight_method_array:
    if eta_reweight == 0:
        nevents_eta = []
        for i in range(6):
            nevents_eta.append(np.ones(3))
    

    if eta_reweight == 1:
        nevents_eta = np.load("../mc_eta_events.npy")
        
    if eta_reweight == 2:
        nevents_eta = []
        for i in range(6):
            nevents_eta.append([1,2,3]) 
        
    if eta_reweight == 3:
        nevents_eta = []
        for i in range(6):
            nevents_eta.append([20,10,1])  
    
    
    
    for k in range(0,6):  #for only dijet event, start frTom jet pT>500 GeV

            
            c = TCanvas("c","c",500,500)
            min = bin[k]
            max = bin[k+1]
            print(min)
            
            nominal_jets =  ReadingDijetMCEta(fdijetMC,str(min),var,"0","0")
            
            data_jets = ReadingDijetDataEta(fdijetData,str(min),var,"0","0")

                    
            higher_quark = nominal_jets["hq"]
            lower_quark = nominal_jets["lq"]
            higher_gluon = nominal_jets["hg"]
            lower_gluon = nominal_jets["lg"]
            higher_other = nominal_jets["ho"]
            lower_other = nominal_jets["lo"]        
            higher_mc = nominal_jets["hmc"]
            lower_mc = nominal_jets["lmc"]        

            
            higher_data_TH1I = data_jets["hd"]
            higher_data = higher_quark.Clone(higher_data_TH1I.GetName()+"TH1F")
            for j in range(1,higher_data.GetNbinsX()+1):
                    higher_data.SetBinContent(j,float(higher_data_TH1I.GetBinContent(j)))
                    higher_data.SetBinError(j,float(higher_data_TH1I.GetBinError(j)))
                    
                    
            lower_data_TH1I = data_jets["ld"]
            lower_data = lower_quark.Clone(lower_data_TH1I.GetName()+"TH1F")
            for j in range(1,higher_data.GetNbinsX()+1):
                    lower_data.SetBinContent(j,float(lower_data_TH1I.GetBinContent(j)))
                    lower_data.SetBinError(j,float(lower_data_TH1I.GetBinError(j))) 
                    
            higher_quark.Scale(nevents_eta[k][0])
            higher_gluon.Scale(nevents_eta[k][0])
            higher_other.Scale(nevents_eta[k][0])
            higher_mc.Scale(nevents_eta[k][0])
            lower_quark.Scale(nevents_eta[k][0])
            lower_gluon.Scale(nevents_eta[k][0])
            lower_other.Scale(nevents_eta[k][0])
            lower_mc.Scale(nevents_eta[k][0])
            higher_data.Scale(nevents_eta[k][0])
            lower_data.Scale(nevents_eta[k][0])
            
            for e in range(1,len(eta_bin)-1):
        
                    print(var,str(eta_bin[e]),nevents_eta[k][e])
                    
                    nominal_jets_add =  ReadingDijetMCEta(fdijetMC,str(min),var,str(eta_bin[e]),str(eta_bin[e]))
                    # flatten the eta distribution
                        
                    data_jets_add = ReadingDijetDataEta(fdijetData,str(min),var,str(eta_bin[e]),str(eta_bin[e]))
                    
                    higher_data_TH1I_add = data_jets_add["hd"]
                    higher_data_add = higher_quark.Clone(higher_data_TH1I_add.GetName()+"TH1F")
                    for j in range(1,higher_data.GetNbinsX()+1):
                            higher_data_add.SetBinContent(j,float(higher_data_TH1I_add.GetBinContent(j)))
                            higher_data_add.SetBinError(j,float(higher_data_TH1I_add.GetBinError(j)))
                            
                            
                    lower_data_TH1I_add = data_jets_add["ld"]
                    lower_data_add = lower_quark.Clone(lower_data_TH1I_add.GetName()+"TH1F")
                    for j in range(1,higher_data.GetNbinsX()+1):
                            lower_data_add.SetBinContent(j,float(lower_data_TH1I_add.GetBinContent(j)))
                            lower_data_add.SetBinError(j,float(lower_data_TH1I_add.GetBinError(j)))   
                            
                    nominal_jets_add["hq"].Scale(nevents_eta[k][e])
                    nominal_jets_add["hg"].Scale(nevents_eta[k][e])
                    nominal_jets_add["ho"].Scale(nevents_eta[k][e])
                    nominal_jets_add["hmc"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lq"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lg"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lo"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lmc"].Scale(nevents_eta[k][e])
                    higher_data_add.Scale(nevents_eta[k][e])
                    lower_data_add.Scale(nevents_eta[k][e])   
                    
                    higher_quark.Add(nominal_jets_add["hq"])
                    lower_quark.Add(nominal_jets_add["lq"])
                    higher_gluon.Add(nominal_jets_add["hg"])
                    lower_gluon.Add(nominal_jets_add["lg"])
                    higher_other.Add(nominal_jets_add["ho"])
                    lower_other.Add(nominal_jets_add["lo"])        
                    higher_mc.Add(nominal_jets_add["hmc"])
                    lower_mc.Add(nominal_jets_add["lmc"])                    
                    higher_data.Add(higher_data_add)
                    lower_data.Add(lower_data_add) 
                    
            higher_quark_unc = unc_array(higher_quark)
            higher_gluon_unc = unc_array(higher_gluon)
            lower_quark_unc = unc_array(lower_quark)
            lower_gluon_unc = unc_array(lower_gluon)
            higher_mc_unc = unc_array(higher_mc)
            lower_mc_unc = unc_array(lower_mc)
            higher_data_unc = unc_array(higher_data)
            lower_data_unc = unc_array(lower_data)
            
            ToT_Fq2_unc = higher_quark_unc.sum()
            ToT_Fg2_unc = higher_gluon_unc.sum()
    
            ToT_Cq2_unc = lower_quark_unc.sum()
            ToT_Cg2_unc = lower_gluon_unc.sum()
    
            # calculate the fraction of forward(higher) / central(lower) quark or gluon jet
            fg_unc=ToT_Fg2_unc/(ToT_Fg2_unc+ToT_Fq2_unc)
            cg_unc=ToT_Cg2_unc/(ToT_Cq2_unc+ToT_Cg2_unc)
            fq_unc=1.-fg_unc
            cq_unc=1.-cg_unc
            
            print(fg_unc-cg_unc)
            
            factor_quark_unc = lower_quark_unc
            factor_gluon_unc = lower_gluon_unc
            
            
            higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
            higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
            lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
            lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()
            higher_data_unc = higher_data_unc/higher_data_unc.sum()
            lower_data_unc = lower_data_unc/lower_data_unc.sum()
            higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
            lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()        
            
            if (doreweight=="Quark"):
                    for i in range(1,higher_quark.GetNbinsX()+1):
                            if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                    factor_gluon_unc[i-1] = higher_gluon_unc[i-1]/lower_gluon_unc[i-1]
                                    factor_quark_unc[i-1] = higher_quark_unc[i-1]/lower_quark_unc[i-1]
                            else:
                                    factor_gluon_unc[i-1] = unc.ufloat(1, 0)
                                    factor_quark_unc[i-1] = unc.ufloat(1, 0)
                    lower_quark_unc=lower_quark_unc*factor_quark_unc
                    lower_gluon_unc=lower_gluon_unc*factor_quark_unc
                    lower_mc_unc=lower_mc_unc*factor_quark_unc
                    lower_data_unc=lower_data_unc*factor_quark_unc
    
            if (doreweight=="Gluon"):
                    for i in range(1,higher_quark.GetNbinsX()+1):
                            if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                    #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                    factor_gluon_unc[i-1] = higher_gluon_unc[i-1]/lower_gluon_unc[i-1]
                                    factor_quark_unc[i-1] = higher_quark_unc[i-1]/lower_quark_unc[i-1]
                            else:
                                    factor_gluon_unc[i-1] = unc.ufloat(1, 0)
                                    factor_quark_unc[i-1] = unc.ufloat(1, 0)
                    lower_quark_unc=lower_quark_unc*factor_gluon_unc
                    lower_gluon_unc=lower_gluon_unc*factor_gluon_unc
                    lower_mc_unc=lower_mc_unc*factor_gluon_unc
                    lower_data_unc=lower_data_unc*factor_gluon_unc
                
            higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
            higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
            lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
            lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()
            higher_data_unc = higher_data_unc/higher_data_unc.sum()
            lower_data_unc = lower_data_unc/lower_data_unc.sum()
            higher_mc_unc = higher_mc_unc/higher_mc_unc.sum()
            lower_mc_unc = lower_mc_unc/lower_mc_unc.sum()   
            
            higher = higher_mc.Clone()
            lower = lower_mc.Clone()
    
            higher_unc = higher_mc_unc
            lower_unc  = lower_mc_unc
    
            
            # matrix method here
            #Now, let's solve.
            F_unc = higher_unc
            C_unc = lower_unc
            Q_unc = -(C_unc*fg_unc-F_unc*cg_unc)/(cg_unc*fq_unc-fg_unc*cq_unc)
            G_unc = (C_unc*fq_unc-F_unc*cq_unc)/(cg_unc*fq_unc-fg_unc*cq_unc)
    
            F_data_unc = higher_data_unc
            C_data_unc = lower_data_unc
            Q_data_unc = -(C_data_unc*fg_unc-F_data_unc*cg_unc)/(cg_unc*fq_unc-fg_unc*cq_unc)
            G_data_unc = (C_data_unc*fq_unc-F_data_unc*cq_unc)/(cg_unc*fq_unc-fg_unc*cq_unc)
    
            quark = higher_quark.Clone()
            gluon = higher_gluon.Clone()
            quark_data = higher_quark.Clone()
            gluon_data = higher_gluon.Clone()
    

            
            # set error bar in histogram  
            # extracted result
            set_hist(quark,Q_unc)
            set_hist(gluon,G_unc)
            set_hist(quark_data,Q_data_unc)
            set_hist(gluon_data,G_data_unc)
            
            # kinematics
            set_hist(higher_quark,higher_quark_unc)
            set_hist(higher_gluon,higher_gluon_unc)
            set_hist(lower_quark,lower_quark_unc)
            set_hist(lower_gluon,lower_gluon_unc)   
            
            #data/mc comparision
            set_hist(higher_data,higher_data_unc)
            set_hist(higher_mc,higher_mc_unc)
            set_hist(lower_data,lower_data_unc)
            set_hist(lower_mc,lower_mc_unc)    
            
            #ratio in Data/MC
            factor_higher_unc = lower_quark_unc
            factor_lower_unc = lower_quark_unc
    
            factor_higher_hist = higher_data.Clone()
            factor_lower_hist = lower_data.Clone()      
    
            factor_higher_hist.Divide(higher_mc)
            factor_lower_hist.Divide(lower_mc)                    
            
            higher_quark_hist = higher_quark.Clone()
            higher_gluon_hist = higher_quark.Clone() 
            lower_quark_hist = higher_quark.Clone()
            lower_gluon_hist = higher_quark.Clone()

            
            for i in range(1,higher_quark.GetNbinsX()+1):
                higher_quark_hist.SetBinContent(i,higher_quark_unc[i-1].nominal_value)
                higher_quark_hist.SetBinError(i,higher_quark_unc[i-1].std_dev)
                lower_quark_hist.SetBinContent(i,lower_quark_unc[i-1].nominal_value)
                lower_quark_hist.SetBinError(i,lower_quark_unc[i-1].std_dev)
                higher_gluon_hist.SetBinContent(i,higher_gluon_unc[i-1].nominal_value)
                higher_gluon_hist.SetBinError(i,higher_gluon_unc[i-1].std_dev)
                lower_gluon_hist.SetBinContent(i,lower_gluon_unc[i-1].nominal_value)
                lower_gluon_hist.SetBinError(i,lower_gluon_unc[i-1].std_dev)
                
            factor_quark_hist = higher_quark.Clone()
            factor_gluon_hist = higher_gluon.Clone()
            for i in range(1,higher_quark.GetNbinsX()+1):

                    factor_quark_hist.SetBinContent(i,factor_quark_unc[i-1].nominal_value)
                    factor_quark_hist.SetBinError(i,factor_quark_unc[i-1].std_dev)
                    factor_gluon_hist.SetBinContent(i,factor_gluon_unc[i-1].nominal_value)
                    factor_gluon_hist.SetBinError(i,factor_gluon_unc[i-1].std_dev)
            
            ## Do the ploting for extracted Data/MC
    
            gPad.SetLeftMargin(0.15)
            gPad.SetTopMargin(0.05)
            gPad.SetBottomMargin(0.15)
    
            quark_ratio = quark.Clone()
            gluon_ratio = gluon.Clone()
            ex_quark_ratio = quark_data.Clone()
            ex_gluon_ratio = gluon_data.Clone()
            ex_quark_ratio.GetYaxis().SetTitle("Data/MC")
            ex_gluon_ratio.GetYaxis().SetTitle("Data/MC")
    
            ex_quark_ratio.Divide(quark)
            ex_gluon_ratio.Divide(gluon)
            quark_ratio.Divide(higher_quark)
            gluon_ratio.Divide(higher_gluon)
            
    
            gStyle.SetOptStat(0)
            ######################## for ratio plot
            c.Divide(2,1)
    
            top = c.cd(1)
            top.SetPad(0.0,0.0,1.0,1.0)
            top.SetFillColor(0)
            top.SetBorderMode(0)
            top.SetBorderSize(2)
            top.SetTickx(1)
            top.SetTicky(1)
            top.SetLeftMargin(0.14)
            top.SetRightMargin(0.055)
            top.SetBottomMargin(0.3)#0.25
            top.SetFrameBorderMode(0)
            #top.SetLogy(1)
    
            bot = c.cd(2)
            bot.SetPad(0.0,0.0,1.0,0.3)
            bot.SetFillColor(0)
            bot.SetBorderMode(0)
            bot.SetBorderSize(2)
            bot.SetTickx(1)
            bot.SetTicky(1)
            bot.SetLeftMargin(0.14)
            bot.SetRightMargin(0.055)
            bot.SetTopMargin(0.045)
            bot.SetBottomMargin(0.4)
            bot.SetFrameBorderMode(0)
    
            quark.SetTitle("")
            quark.GetXaxis().SetTitle(var)
            quark.GetYaxis().SetTitle("Normalized to unity")
            quark.GetYaxis().SetNdivisions(505)
            quark.GetYaxis().SetRangeUser(-0.01,quark.GetMaximum()*1.5)
    
            quark.SetMarkerColor(4)
            quark.SetLineColor(4)
            quark.SetMarkerSize(0.5)
            quark.SetMarkerStyle(20)
    
            quark_data.SetMarkerColor(1)
            quark_data.SetLineColor(1)
            quark_data.SetMarkerSize(0.5)
            quark_data.SetMarkerStyle(8)
    
            higher_quark.SetMarkerColor(4)
            higher_quark.SetLineColor(4)
            higher_quark.SetMarkerSize(0.5)
            higher_quark.SetLineStyle(2)
    
            ex_quark_ratio.SetTitle("")
            ex_quark_ratio.GetYaxis().SetRangeUser(0.7,1.3)
            ex_quark_ratio.GetXaxis().SetTitleOffset(1)
            ex_quark_ratio.GetXaxis().SetTitleSize(0.11)
            ex_quark_ratio.GetXaxis().SetLabelSize(0.1)
            ex_quark_ratio.GetXaxis().SetLabelOffset(0.03)
            ex_quark_ratio.GetYaxis().SetTitleSize(0.1)
            ex_quark_ratio.GetYaxis().SetTitleOffset(0.5)
            ex_quark_ratio.GetYaxis().SetLabelOffset(0.01)
            ex_quark_ratio.GetYaxis().SetLabelSize(0.1)
            ex_quark_ratio.GetYaxis().SetNdivisions(505)
    
            quark_ratio.SetMarkerColor(4)
            quark_ratio.SetLineColor(4)
            quark_ratio.SetMarkerSize(0.5)
            quark_ratio.SetMarkerStyle(20)
    
            ex_quark_ratio.SetMarkerColor(4)
            ex_quark_ratio.SetLineColor(4)
            ex_quark_ratio.SetMarkerSize(0.5)
            ex_quark_ratio.SetLineStyle(20)
    
            top.cd()
    
            quark.Draw("HIST") #extracted mc
            quark_data.Draw("same") #extracted mc
            #higher_quark.Draw("HIST same") #truth mc
    
            if(var == "ntrk"):
                line = TLine(0.,1,60,1)
                ex_quark_ratio.GetXaxis().SetTitle("N_{track}")
                leg = TLegend(0.6,0.5,0.9,0.7)
            if(var == "bdt"):
                line = TLine(-0.8,1,0.7,1)
                ex_quark_ratio.GetXaxis().SetTitle("BDT score")
                leg = TLegend(0.6,0.65,0.9,0.85) ##0.6,0.5,0.9,0.7
    
            leg.SetTextFont(42)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            leg.SetFillStyle(0)
            leg.SetNColumns(1)
            leg.AddEntry(quark_data,"Extracted quark (data)","p")
            leg.AddEntry(quark,"Extracted quark (mc)","l")
            #leg.AddEntry(higher_quark,"Quark (mc)","l")
            leg.Draw()
    
            myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
            myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
            myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
    
            bot.cd()
            #quark_ratio.Draw("HIST")
            ex_quark_ratio.SetFillColor(4)
            ex_quark_ratio.SetFillColor(4)
            ex_quark_ratio.SetFillStyle(3005)
            ex_quark_ratio.Draw("e2")
            line.Draw("same")
            c.Print("./plots_"+var+"/quark_"+str(min)+"-"+str(max)+"_"+str(doreweight)+"_"+mc+"_"+var+"_"+str(eta_reweight)+".pdf")
    
            gluon.SetTitle("")
            gluon.GetXaxis().SetTitle(var)
            gluon.GetYaxis().SetTitle("Normalized to unity")
            gluon.GetYaxis().SetNdivisions(505)
            gluon.GetYaxis().SetRangeUser(-0.01,gluon.GetMaximum()*1.5)
    
            gluon.SetMarkerColor(2)
            gluon.SetLineColor(2)
            gluon.SetMarkerSize(0.5)
            gluon.SetMarkerStyle(20)
    
            gluon_data.SetMarkerColor(1)
            gluon_data.SetLineColor(1)
            gluon_data.SetMarkerSize(0.5)
            gluon_data.SetMarkerStyle(8)
    
            higher_gluon.SetMarkerColor(2)
            higher_gluon.SetLineColor(2)
            higher_gluon.SetMarkerSize(0.5)
            higher_gluon.SetLineStyle(2)
    
            ex_gluon_ratio.SetTitle("")
            ex_gluon_ratio.GetYaxis().SetRangeUser(0.7,1.3)
            ex_gluon_ratio.GetXaxis().SetTitleOffset(1)
            ex_gluon_ratio.GetXaxis().SetTitleSize(0.11)
            ex_gluon_ratio.GetXaxis().SetLabelSize(0.1)
            ex_gluon_ratio.GetXaxis().SetLabelOffset(0.03)
            ex_gluon_ratio.GetYaxis().SetTitleSize(0.1)
            ex_gluon_ratio.GetYaxis().SetTitleOffset(0.5)
            ex_gluon_ratio.GetYaxis().SetLabelOffset(0.01)
            ex_gluon_ratio.GetYaxis().SetLabelSize(0.1)
            ex_gluon_ratio.GetYaxis().SetNdivisions(505)
            
    
            ex_gluon_ratio.SetMarkerColor(2)
            ex_gluon_ratio.SetLineColor(2)
            ex_gluon_ratio.SetMarkerSize(0.5)
            ex_gluon_ratio.SetLineStyle(20)
    
            top.cd()
            gluon.Draw("HIST")
            gluon_data.Draw("same")
            #higher_gluon.Draw("HIST same")
            
            if(var == "ntrk"):
                line = TLine(0.,1,60,1)
                ex_gluon_ratio.GetXaxis().SetTitle("N_{track}")
                leg = TLegend(0.6,0.5,0.9,0.7)
            if(var == "bdt"):
                line = TLine(-0.8,1,0.7,1)
                ex_gluon_ratio.GetXaxis().SetTitle("BDT score")
                leg = TLegend(0.2,0.5,0.5,0.7) ##0.6,0.5,0.9,0.7
    
            leg.SetTextFont(42)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            leg.SetFillStyle(0)
            leg.SetNColumns(1)
            leg.AddEntry(gluon_data,"Extracted gluon (data)","p")
            leg.AddEntry(gluon,"Extracted gluon (mc)","l")
            #leg.AddEntry(higher_gluon,"Gluon (mc)","l")
            leg.Draw()
    
            myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
            myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
            myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
            bot = c.cd(2)
            bot.SetPad(0.0,0.0,1.0,0.3)
            bot.SetFillColor(0)
            bot.SetBorderMode(0)
            bot.SetBorderSize(2)
            bot.SetTickx(1)
            bot.SetTicky(1)
            bot.SetLeftMargin(0.14)
            bot.SetRightMargin(0.055)
            bot.SetTopMargin(0.045)
            bot.SetBottomMargin(0.4)
            bot.SetFrameBorderMode(0)
            bot.cd()
            #gluon_ratio.Draw("HIST")
            ex_gluon_ratio.SetMarkerSize(2)
            ex_gluon_ratio.SetFillColor(2)
            ex_gluon_ratio.SetFillStyle(3005)
            ex_gluon_ratio.Draw("e2")
            line.Draw("same")
            c.Print("./plots_"+var+"/gluon_"+str(min)+"-"+str(max)+"_"+str(doreweight)+"_"+mc+"_"+var+"_"+str(eta_reweight)+".pdf")
            
            
            ## Do the ploting for extracted MC closure
            c2 = TCanvas("","",500,500)
            gPad.SetLeftMargin(0.15)
            gPad.SetTopMargin(0.05)
            gPad.SetBottomMargin(0.15)
            quark_ratio = quark.Clone("")
            gluon_ratio = gluon.Clone("")
            quark_ratio.GetYaxis().SetTitle("Extracted/Truth")
            gluon_ratio.GetYaxis().SetTitle("Extracted/Truth")
            ex_quark_ratio.Divide(quark)
            ex_gluon_ratio.Divide(gluon)
            quark_ratio.Divide(higher_quark)
            gluon_ratio.Divide(higher_gluon)
            gStyle.SetOptStat(0)
            ######################## for ratio plot
            c2.Divide(2,1)
            top = c2.cd(1)
            top.SetPad(0.0,0.0,1.0,1.0)
            top.SetFillColor(0)
            top.SetBorderMode(0)
            top.SetBorderSize(2)
            top.SetTickx(1)
            top.SetTicky(1)
            top.SetLeftMargin(0.14)
            top.SetRightMargin(0.055)
            top.SetBottomMargin(0.3)#0.25
            top.SetFrameBorderMode(0)
            #top.SetLogy(1)
            bot = c2.cd(2)
            bot.SetPad(0.0,0.0,1.0,0.3)
            bot.SetFillColor(0)
            bot.SetBorderMode(0)
            bot.SetBorderSize(2)
            bot.SetTickx(1)
            bot.SetTicky(1)
            bot.SetLeftMargin(0.14)
            bot.SetRightMargin(0.055)
            bot.SetTopMargin(0.045)
            bot.SetBottomMargin(0.4)
            bot.SetFrameBorderMode(0)
            quark.SetTitle("")
            quark.GetXaxis().SetTitle(var)
            quark.GetYaxis().SetTitle("Normalized to unity")
            quark.GetYaxis().SetNdivisions(505)
            quark.GetYaxis().SetRangeUser(-0.01,quark.GetMaximum()*1.5)
            quark.SetMarkerColor(4)
            quark.SetLineColor(4)
            quark.SetMarkerSize(0.5)
            quark.SetMarkerStyle(20)
    
            quark_data.SetMarkerColor(1)
            quark_data.SetLineColor(1)
            quark_data.SetMarkerSize(0.5)
            quark_data.SetMarkerStyle(8)
    
            higher_quark.SetMarkerColor(4)
            higher_quark.SetLineColor(4)
            higher_quark.SetMarkerSize(0.5)
            higher_quark.SetLineStyle(2)
            quark_ratio.SetTitle("")
            quark_ratio.GetYaxis().SetRangeUser(0.7,1.3)
            quark_ratio.GetXaxis().SetTitleOffset(1)
            quark_ratio.GetXaxis().SetTitleSize(0.11)
            quark_ratio.GetXaxis().SetLabelSize(0.1)
            quark_ratio.GetXaxis().SetLabelOffset(0.03)
            quark_ratio.GetYaxis().SetTitleSize(0.1)
            quark_ratio.GetYaxis().SetTitleOffset(0.5)
            quark_ratio.GetYaxis().SetLabelOffset(0.01)
            quark_ratio.GetYaxis().SetLabelSize(0.1)
            quark_ratio.SetMarkerColor(4)
            quark_ratio.SetLineColor(4)
            quark_ratio.SetMarkerSize(0.5)
            quark_ratio.SetMarkerStyle(20)
            top.cd()
            quark.Draw("HIST") #extracted mc
            higher_quark.Draw("HIST same") #truth mc
            if(var == "c1"):
    #                   line = TLine(0.,1,60,1)
                line = TLine(quark_ratio.GetXaxis().GetXmin(), 1.0, quark_ratio.GetXaxis().GetXmax(),1.0)
                quark_ratio.GetXaxis().SetTitle("C_{1}")
                leg = TLegend(0.6,0.5,0.9,0.7)
            if(var == "width"):
    #                   line = TLine(-0.8,1,0.7,1)
                line = TLine(quark_ratio.GetXaxis().GetXmin(), 1.0, quark_ratio.GetXaxis().GetXmax(),1.0)
                quark_ratio.GetXaxis().SetTitle("W_{track}")
                leg = TLegend(0.2,0.5,0.5,0.7) ##0.6,0.5,0.9,0.7
            if(var == "eta"):
    #                   line = TLine(-0.8,1,0.7,1)
                line = TLine(quark_ratio.GetXaxis().GetXmin(), 1.0, quark_ratio.GetXaxis().GetXmax(),1.0)
                quark_ratio.GetXaxis().SetTitle("#eta")
                leg = TLegend(0.2,0.5,0.5,0.7) ##0.6,0.5,0.9,0.7    
            if(var == "ntrk"):
    #                   line = TLine(0.,1,60,1)
                line = TLine(quark_ratio.GetXaxis().GetXmin(), 1.0, quark_ratio.GetXaxis().GetXmax(),1.0)
                quark_ratio.GetXaxis().SetTitle("N_{track}")
                leg = TLegend(0.6,0.5,0.9,0.7)
            if(var == "bdt"):
    #                   line = TLine(-0.8,1,0.7,1)
                line = TLine(quark_ratio.GetXaxis().GetXmin(), 1.0, quark_ratio.GetXaxis().GetXmax(),1.0)
                quark_ratio.GetXaxis().SetTitle("BDT value")
                leg = TLegend(0.6,0.65,0.9,0.85) ##0.6,0.5,0.9,0.7
                
            leg.SetTextFont(42)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            leg.SetFillStyle(0)
            leg.SetNColumns(1)
            #leg.AddEntry(quark_data,"Extracted quark (data)","p")
            leg.AddEntry(quark,"Extracted quark (mc)","l")
            leg.AddEntry(higher_quark,"Quark (mc)","l")
            leg.Draw()
            myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
            myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
            myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
            bot.cd()
            quark_ratio.SetFillColor(4)
            quark_ratio.SetFillStyle(3005)
            quark_ratio.Draw("e2")
            #ex_quark_ratio.Draw("same HIST")
            line.Draw("same")
            c2.Print(path + "/quark_"+str(min)+"-"+str(max)+"_"+str(doreweight)+"_"+mc+"_"+var+"_"+str(eta_reweight)+".pdf")
            gluon.SetTitle("")
            gluon.GetXaxis().SetTitle(var)
            gluon.GetYaxis().SetTitle("Normalized to unity")
            gluon.GetYaxis().SetNdivisions(505)
            gluon.GetYaxis().SetRangeUser(-0.01,gluon.GetMaximum()*1.5)
            gluon.SetMarkerColor(2)
            gluon.SetLineColor(2)
            gluon.SetLineStyle(1)
            gluon.SetMarkerSize(0.5)
            gluon.SetMarkerStyle(20)
            gluon_data.SetMarkerColor(1)
            gluon_data.SetLineColor(1)
            gluon_data.SetMarkerSize(0.5)
            gluon_data.SetMarkerStyle(8)
            higher_gluon.SetMarkerColor(2)
            higher_gluon.SetLineColor(2)
            higher_gluon.SetMarkerSize(0.5)
            higher_gluon.SetLineStyle(2)
            gluon_ratio.SetTitle("")
            gluon_ratio.GetYaxis().SetRangeUser(0.7,1.3)
            gluon_ratio.GetXaxis().SetTitleOffset(1)
            gluon_ratio.GetXaxis().SetTitleSize(0.11)
            gluon_ratio.GetXaxis().SetLabelSize(0.1)
            gluon_ratio.GetXaxis().SetLabelOffset(0.03)
            gluon_ratio.GetYaxis().SetTitleSize(0.1)
            gluon_ratio.GetYaxis().SetTitleOffset(0.5)
            gluon_ratio.GetYaxis().SetLabelOffset(0.01)
            gluon_ratio.GetYaxis().SetLabelSize(0.1)
            gluon_ratio.SetMarkerColor(2)
            gluon_ratio.SetLineColor(2)
            gluon_ratio.SetMarkerSize(0.5)
            gluon_ratio.SetMarkerStyle(20)
            top.cd()
            gluon.Draw("HIST")
            #gluon_data.Draw("same")
            higher_gluon.Draw("HIST same")
            if(var == "c1"):
    #                   line = TLine(0.,1,60,1)
                line = TLine(gluon_ratio.GetXaxis().GetXmin(), 1.0, gluon_ratio.GetXaxis().GetXmax(),1.0)
                gluon_ratio.GetXaxis().SetTitle("C_{1}")
                leg = TLegend(0.6,0.5,0.9,0.7)
            if(var == "width"):
    #                   line = TLine(-0.8,1,0.7,1)
                line = TLine(gluon_ratio.GetXaxis().GetXmin(), 1.0, gluon_ratio.GetXaxis().GetXmax(),1.0)
                gluon_ratio.GetXaxis().SetTitle("W_{track}")
                leg = TLegend(0.2,0.5,0.5,0.7) ##0.6,0.5,0.9,0.7
            if(var == "eta"):
    #                   line = TLine(-0.8,1,0.7,1)
                line = TLine(gluon_ratio.GetXaxis().GetXmin(), 1.0, gluon_ratio.GetXaxis().GetXmax(),1.0)
                gluon_ratio.GetXaxis().SetTitle("#eta")
                leg = TLegend(0.2,0.5,0.5,0.7) ##0.6,0.5,0.9,0.7            
            if(var == "ntrk"):
    #                   line = TLine(0.,1,60,1)
                line = TLine(gluon_ratio.GetXaxis().GetXmin(), 1.0, gluon_ratio.GetXaxis().GetXmax(),1.0)
                gluon_ratio.GetXaxis().SetTitle("N_{track}")
                leg = TLegend(0.6,0.5,0.9,0.7)
            if(var == "bdt"):
    #                   line = TLine(-0.8,1,0.7,1)
                line = TLine(gluon_ratio.GetXaxis().GetXmin(), 1.0, gluon_ratio.GetXaxis().GetXmax(),1.0)
                gluon_ratio.GetXaxis().SetTitle("BDT score")
                leg = TLegend(0.2,0.5,0.5,0.7) ##0.6,0.5,0.9,0.7
            leg.SetTextFont(42)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            leg.SetFillStyle(0)
            leg.SetNColumns(1)
            #leg.AddEntry(gluon_data,"Extracted gluon (data)","p")
            leg.AddEntry(gluon,"Extracted gluon (mc)","l")
            leg.AddEntry(higher_gluon,"Gluon (mc)","l")
            leg.Draw()
            myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
            myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
            myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
            bot.cd()
            gluon_ratio.SetFillColor(2)
            gluon_ratio.SetFillStyle(3005)
            gluon_ratio.Draw("e2")
            #ex_gluon_ratio.Draw("same HIST")
            line.Draw("same")
            c2.Print(path + "/gluon_"+str(min)+"-"+str(max)+"_"+str(doreweight)+"_"+mc+"_"+var+"_"+str(eta_reweight)+".pdf")
    
        
            c3 = TCanvas("","",500,500)
            gStyle.SetOptStat(0)
            
            c3.Divide(2,1)
            top = c3.cd(1)
            top.SetPad(0.0,0.0,1.0,1.0)
            top.SetFillColor(0)
            top.SetBorderMode(0)
            top.SetBorderSize(2)
            top.SetTickx(1)
            top.SetTicky(1)
            top.SetLeftMargin(0.14)
            top.SetRightMargin(0.055)
            top.SetBottomMargin(0.3)#0.25
            top.SetFrameBorderMode(0)
            #top.SetLogy(1)
            top.cd()
            higher_mc.SetMarkerColor(4)
            higher_mc.SetLineColor(4)
            higher_mc.SetMarkerSize(0.5)
            higher_mc.SetLineStyle(1)
    
            higher_data.SetMarkerColor(1)
            higher_data.SetLineColor(1)
            higher_data.SetMarkerSize(0.5)
            higher_data.SetLineStyle(2)
    
            bot = c3.cd(2)
            bot.SetPad(0.0,0.0,1.0,0.3)
            bot.SetFillColor(0)
            bot.SetBorderMode(0)
            bot.SetBorderSize(2)
            bot.SetTickx(1)
            bot.SetTicky(1)
            bot.SetLeftMargin(0.14)
            bot.SetRightMargin(0.055)
            bot.SetTopMargin(0.045)
            bot.SetBottomMargin(0.4)
            bot.SetFrameBorderMode(0)        
            factor_higher_hist.SetMarkerColor(4)
            factor_higher_hist.SetLineColor(4)
            factor_higher_hist.SetMarkerSize(0.5)
            factor_higher_hist.SetLineStyle(1)
    
            #leg.AddEntry(gluon_data,"Extracted gluon (data)","p")
            factor_higher_hist.SetMinimum(0)
            factor_higher_hist.SetMaximum(2)
            factor_higher_hist.GetYaxis().SetTitle("Data/MC")        
            factor_higher_hist.GetYaxis().SetRangeUser(0.7,1.3)
            factor_higher_hist.GetXaxis().SetTitleOffset(1)
            factor_higher_hist.GetXaxis().SetTitleSize(0.11)
            factor_higher_hist.GetXaxis().SetLabelSize(0.1)
            factor_higher_hist.GetXaxis().SetLabelOffset(0.03)
            factor_higher_hist.GetYaxis().SetTitleSize(0.1)
            factor_higher_hist.GetYaxis().SetTitleOffset(0.5)
            factor_higher_hist.GetYaxis().SetLabelOffset(0.01)
            factor_higher_hist.GetYaxis().SetLabelSize(0.1)
            top.cd()
            rmax = higher_gluon.GetMaximum()*1.5
            higher_mc.GetYaxis().SetTitle("Normalized to unity")        
            higher_mc.SetMaximum(rmax)
            higher_mc.Draw("HIST ")
            higher_data.Draw("P0 same")
            if(var == "ntrk"):
                       factor_higher_hist.GetXaxis().SetTitle("N_{track}")
 
            if(var == "bdt"):
                       factor_higher_hist.GetXaxis().SetTitle("BDT")
  

            leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  
            leg.AddEntry(higher_mc,"Higher MC","l")
            leg.AddEntry(higher_data,"Higher Data","lp")  
            leg.SetTextFont(42)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            leg.SetFillStyle(0)
            leg.SetNColumns(1)
            leg.Draw()        
            myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
            myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
            myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
            bot.cd()
            factor_higher_hist.Draw("HIST e")
            c3.Print(path +"/" + var+"_"+str(min)+"_"+str(doreweight)+"_"+mc+"_"+var+"_"+str(eta_reweight)+"_dataMC_Higher.png")
            
            
            c4 = TCanvas("","",500,500)
            gStyle.SetOptStat(0)
            
            c4.Divide(2,1)
            top = c4.cd(1)
            top.SetPad(0.0,0.0,1.0,1.0)
            top.SetFillColor(0)
            top.SetBorderMode(0)
            top.SetBorderSize(2)
            top.SetTickx(1)
            top.SetTicky(1)
            top.SetLeftMargin(0.14)
            top.SetRightMargin(0.055)
            top.SetBottomMargin(0.3)#0.25
            top.SetFrameBorderMode(0)
            #top.SetLogy(1)
            top.cd()
    
            lower_mc.SetMarkerColor(2)
            lower_mc.SetLineColor(2)
            lower_mc.SetMarkerSize(0.5)
            lower_mc.SetLineStyle(1)
            lower_data.SetMarkerColor(1)
            lower_data.SetLineColor(1)
            lower_data.SetMarkerSize(0.5)
            lower_data.SetLineStyle(2)
    
            bot = c4.cd(2)
            bot.SetPad(0.0,0.0,1.0,0.3)
            bot.SetFillColor(0)
            bot.SetBorderMode(0)
            bot.SetBorderSize(2)
            bot.SetTickx(1)
            bot.SetTicky(1)
            bot.SetLeftMargin(0.14)
            bot.SetRightMargin(0.055)
            bot.SetTopMargin(0.045)
            bot.SetBottomMargin(0.4)
            bot.SetFrameBorderMode(0)        
    
            factor_lower_hist.SetMarkerColor(2)
            factor_lower_hist.SetLineColor(2)
            factor_lower_hist.SetMarkerSize(0.5)
            factor_lower_hist.SetLineStyle(1)
            #leg.AddEntry(gluon_data,"Extracted gluon (data)","p")
            factor_lower_hist.SetMinimum(0)
            factor_lower_hist.SetMaximum(2)
            factor_lower_hist.GetYaxis().SetTitle("Data/MC")        
            factor_lower_hist.GetYaxis().SetRangeUser(0.7,1.3)
            factor_lower_hist.GetXaxis().SetTitleOffset(1)
            factor_lower_hist.GetXaxis().SetTitleSize(0.11)
            factor_lower_hist.GetXaxis().SetLabelSize(0.1)
            factor_lower_hist.GetXaxis().SetLabelOffset(0.03)
            factor_lower_hist.GetYaxis().SetTitleSize(0.1)
            factor_lower_hist.GetYaxis().SetTitleOffset(0.5)
            factor_lower_hist.GetYaxis().SetLabelOffset(0.01)
            factor_lower_hist.GetYaxis().SetLabelSize(0.1)
            top.cd()
            rmax = lower_gluon.GetMaximum()*1.5
            lower_mc.GetYaxis().SetTitle("Normalized to unity")        
            lower_mc.SetMaximum(rmax)
            lower_mc.Draw("HIST ")
            lower_data.Draw("P0 same")
            if(var == "ntrk"):
                       factor_lower_hist.GetXaxis().SetTitle("N_{track}")
 
            if(var == "bdt"):
                       factor_lower_hist.GetXaxis().SetTitle("BDT")  
            leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  
            leg.AddEntry(lower_mc,"Lower MC","l")
            leg.AddEntry(lower_data,"Lower Data","lp")  
            leg.SetTextFont(42)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            leg.SetFillStyle(0)
            leg.SetNColumns(1)
            leg.Draw()        
            myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
            myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
            myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
            bot.cd()
            factor_lower_hist.Draw("HIST e")
            c4.Print(path +"/"+ var+"_"+str(min)+"_"+str(doreweight)+"_"+mc+"_"+var+"_"+str(eta_reweight)+"_dataMC_Lower.png")
            
            c5 = TCanvas("","",500,500)
            c5.Divide(2,1)

            top = c5.cd(1)
            top.SetPad(0.0,0.0,1.0,1.0)
            top.SetFillColor(0)
            top.SetBorderMode(0)
            top.SetBorderSize(2)
            top.SetTickx(1)
            top.SetTicky(1)
            top.SetLeftMargin(0.14)
            top.SetRightMargin(0.055)
            top.SetBottomMargin(0.3)#0.25
            top.SetFrameBorderMode(0)
            #top.SetLogy(1)
            top.cd()
            higher_quark_hist.SetMarkerColor(4)
            higher_quark_hist.SetLineColor(4)
            higher_quark_hist.SetMarkerSize(0.5)
            higher_quark_hist.SetLineStyle(1)
            higher_gluon_hist.SetMarkerColor(2)
            higher_gluon_hist.SetLineColor(2)
            higher_gluon_hist.SetMarkerSize(0.5)
            higher_gluon_hist.SetLineStyle(1)         
            lower_quark_hist.SetMarkerColor(4)
            lower_quark_hist.SetLineColor(4)
            lower_quark_hist.SetMarkerSize(0.5)
            lower_quark_hist.SetLineStyle(2)
            lower_gluon_hist.SetMarkerColor(2)
            lower_gluon_hist.SetLineColor(2)
            lower_gluon_hist.SetMarkerSize(0.5)
            lower_gluon_hist.SetLineStyle(2)           
            bot = c5.cd(2)
            bot.SetPad(0.0,0.0,1.0,0.3)
            bot.SetFillColor(0)
            bot.SetBorderMode(0)
            bot.SetBorderSize(2)
            bot.SetTickx(1)
            bot.SetTicky(1)
            bot.SetLeftMargin(0.14)
            bot.SetRightMargin(0.055)
            bot.SetTopMargin(0.045)
            bot.SetBottomMargin(0.4)
            bot.SetFrameBorderMode(0)        
            factor_quark_hist.SetMarkerColor(4)
            factor_quark_hist.SetLineColor(4)
            factor_quark_hist.SetMarkerSize(0.5)
            factor_quark_hist.SetLineStyle(1)
            factor_gluon_hist.SetMarkerColor(2)
            factor_gluon_hist.SetLineColor(2)
            factor_gluon_hist.SetMarkerSize(0.5)
            factor_gluon_hist.SetLineStyle(1)   
            #leg.AddEntry(gluon_data,"Extracted gluon (data)","p")
            factor_quark_hist.SetMinimum(0)
            factor_quark_hist.SetMaximum(2)
            factor_quark_hist.GetYaxis().SetTitle("Forward/Central")        
            factor_quark_hist.GetYaxis().SetRangeUser(0.7,1.3)
            factor_quark_hist.GetXaxis().SetTitleOffset(1)
            factor_quark_hist.GetXaxis().SetTitleSize(0.11)
            factor_quark_hist.GetXaxis().SetLabelSize(0.1)
            factor_quark_hist.GetXaxis().SetLabelOffset(0.03)
            factor_quark_hist.GetYaxis().SetTitleSize(0.1)
            factor_quark_hist.GetYaxis().SetTitleOffset(0.5)
            factor_quark_hist.GetYaxis().SetLabelOffset(0.01)
            factor_quark_hist.GetYaxis().SetNdivisions(505)
            
            top.cd()
            rmax = higher_gluon_hist.GetMaximum()*1.5
            higher_quark_hist.GetYaxis().SetTitle("Normalized to unity")        
            higher_quark_hist.SetMaximum(rmax)
            higher_quark_hist.Draw("HIST")
            higher_gluon_hist.Draw("HIST same")
            lower_quark_hist.Draw("HIST same")
            lower_gluon_hist.Draw("HIST same")
            if(var == "ntrk"):
                    factor_quark_hist.GetXaxis().SetTitle("N_{track}")
                    leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  
            if(var == "bdt"):
                    factor_quark_hist.GetXaxis().SetTitle("BDT")
                    leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  

            leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  
            leg.AddEntry(higher_quark_hist,"forward quark","l")
            leg.AddEntry(lower_quark_hist,"central quark","l")     
            leg.AddEntry(higher_gluon_hist,"forward gluon","l")
            leg.AddEntry(lower_gluon_hist,"central gluon","l")  
            leg.SetTextFont(42)
            leg.SetFillColor(0)
            leg.SetBorderSize(0)
            leg.SetFillStyle(0)
            leg.SetNColumns(1)
            leg.Draw()        
            myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
            myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
            myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")
            bot.cd()
            factor_quark_hist.Draw("HIST e")
            factor_gluon_hist.Draw("HIST e same")

            c3.Print(path +"/"+ var+"_"+str(min)+"_"+"_"+mc+"_"+var+"_"+str(eta_reweight)+"_distribution.png")

500
bdt 0.4 1.0
bdt 0.8 1.0
-0.09176+/-0.00032
600
bdt 0.4 1.0
bdt 0.8 1.0
-0.09776+/-0.00027
800
bdt 0.4 1.0
bdt 0.8 1.0
-0.09834+/-0.00025
1000
bdt 0.4 1.0
bdt 0.8 1.0
-0.09849+/-0.00034
1200
bdt 0.4 1.0
bdt 0.8 1.0
-0.09341+/-0.00028
1500
bdt 0.4 1.0
bdt 0.8 1.0
-0.08367+/-0.00027
500
bdt 0.4 1.106436890805779
bdt 0.8 1.9573325300796103
-0.09604+/-0.00034
600
bdt 0.4 1.0901923349709224
bdt 0.8 2.0297953404345015
-0.10259+/-0.00028
800
bdt 0.4 1.0573120288483782
bdt 0.8 2.201884570294298
-0.10499+/-0.00028
1000
bdt 0.4 1.0346283787214403
bdt 0.8 2.398827580762742
-0.1079+/-0.0005
1200
bdt 0.4 1.0097688603329724
bdt 0.8 2.7117476775718448
-0.10630+/-0.00031
1500
bdt 0.4 0.9798242349232528
bdt 0.8 3.404129855454994
-0.10184+/-0.00030
500
bdt 0.4 2
bdt 0.8 3
-0.09199+/-0.00034
600
bdt 0.4 2
bdt 0.8 3
-0.09764+/-0.00028
800
bdt 0.4 2
bdt 0.8 3
-0.09882+/-0.00028
1000
bdt 0.4 2
bdt 0.8 3
-0.1002+/-0.0005
1200
bdt 0.4 2
bdt 0.8 3
-0.09687+/-0.00030
1500
bdt 0.4 2
bdt 0.8 3
-0.08936+/-0.000

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: pdf file ./plots_bdt/quark_500-600_Quark_pythia-eta-test_bdt_0.pdf has been created
Info in <TCanvas::Print>: pdf file ./plots_bdt/gluon_500-600_Quark_pythia-eta-test_bdt_0.pdf has been created
Info in <TCanvas::Print>: pdf file pythia-eta-test_distribution/quark_500-600_Quark_pythia-eta-test_bdt_0.pdf has been created
Info in <TCanvas::Print>: pdf file pythia-eta-test_distribution/gluon_500-600_Quark_pythia-eta-test_bdt_0.pdf has been created
Info in <TCanvas::Print>: png file pythia-eta-test_distribution/bdt_500_Quark_pythia-eta-test_bdt_0_dataMC_Higher.png has been created
Info in <TCanvas::Print>: png file pythia-eta-test_distribution/bdt_500_Quark_pythia-eta-test_bdt_0_dataMC_Lower.png has been created
Info in <TCanvas::Print>: png file pythia-eta-test_distribution/bdt_500__pythia-eta-test_bdt_0_distribution.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with sam

In [3]:
fg_unc

0.20061810627235122+/-0.0001911906146584361

In [4]:
fq_unc

0.7993818937276488+/-0.0001911906146584361

In [1]:
from ROOT import *
import numpy as np
from prettytable import PrettyTable
import sys
import uncertainties as unc # propagate uncertainties
from uncertainties import unumpy as unp # array operations for type ufloat
import os
    
var = "ntrk"  #change the var name according to the var you want to read
mc = "pythia-eta-test"   #by setting it as "SF" or "MC", it will automatically making scale factor plots or MC closure plots

if var == "ntrk":
    doreweight = "Quark"  
if var == "bdt":
    doreweight = "Quark" 
    
eta_bin = [0,0.4,0.8,1.4]

reweight_method_array =[0,1,2,3]




fdijetMC = TFile("/eos/user/r/rqian/dijet-mono-result/pythia-eta-test/dijet_pythia_etatest_1-4.root")

fdijetData = TFile("/eos/user/r/rqian/dijet-mono-result/data-eta-test/dijet_data_1-4.root")
bins = np.array([500.,600.,800.,1000.,1200.,1500.,2000.])
bin = np.array([500,600,800,1000,1200,1500,2000])

path = mc+"_distribution"
isExists=os.path.exists(path)
if not isExists:
        os.makedirs(path)        

var_path = "plots_" + var
isExists=os.path.exists(var_path)
if not isExists:
        os.makedirs(var_path)        

        
#convert histogram and error into unp.uarray
#if sample is pyroot input, GetBinError returns correct result
def unc_array(hist):
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = hist.GetBinError(j)
    result = unp.uarray(value,error)
    return(result)

#convert histogram and error into unp.uarray
#if sample is uproot input, err is the sumw2 of the corresponding histogram
def unc_array_err(hist,err):
    
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = np.sqrt(err.GetBinContent(j))
    result = unp.uarray(value,error)
    return(result)


def myText(x,y,text, color = 1):
    l = TLatex()
    l.SetTextSize(0.025)
    l.SetNDC()
    l.SetTextColor(color)
    l.DrawLatex(x,y,text)
    pass

def error_check(hist):
    for i in range(1,hist.GetNbinsX()+1):
        if hist.GetBinContent(i)>1:
            hist.SetBinContent(i,0)
    return(hist)



# convert uncertainty array into histogram
def set_hist(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))
        hist.SetBinContent(i,unp.nominal_values(unc[i-1]))
        
def set_hist_error(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))

#inclusive fraction calculation
def fraction(lower_quark,lower_gluon,higher_quark,higher_gluon):
    ToT_Fq2 = 0.
    ToT_Fg2 = 0.
    ToT_Cq2 =0.
    ToT_Cg2 = 0.

    for j in range(1,lower_quark.GetNbinsX()+1):
        ToT_Fq2+=higher_quark.GetBinContent(j)
        ToT_Cq2+=lower_quark.GetBinContent(j)
        ToT_Fg2+=higher_gluon.GetBinContent(j)
        ToT_Cg2+=lower_gluon.GetBinContent(j)

    # calculate the fraction of each sample 
    if ((ToT_Fg2+ToT_Fq2) != 0):
        fg=ToT_Fg2/(ToT_Fg2+ToT_Fq2)
        cg=ToT_Cg2/(ToT_Cq2+ToT_Cg2)

    fq=1.-fg
    cq=1.-cg
    return(fg,cg,fq,cq)

def mc_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,fg,cg,fq,cq,higher_mc_input,lower_mc_input):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower_mc = lower_mc_input.Clone()
        higher_mc = higher_mc_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_quark)


            
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower_mc.SetBinContent(i,lower_mc.GetBinContent(i)*factor_gluon)

        
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower_mc.Integral() != 0):
                lower_mc.Scale(1./lower_mc.Integral())
        if(higher_mc.Integral() != 0):
                higher_mc.Scale(1./higher_mc.Integral())

        higher = higher_mc.Clone()
        lower = lower_mc.Clone()          


        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)

#return the abs systematic uncertainty
def mc_error(mc_quark_1,mc_gluon_1,mc_quark_2,mc_gluon_2): 
    error_q = np.zeros(mc_quark_1.GetNbinsX())
    error_g = np.zeros(mc_gluon_1.GetNbinsX())

    for j in range(1,mc_gluon_1.GetNbinsX()+1):
                q = abs(mc_quark_1.GetBinContent(j) - mc_quark_2.GetBinContent(j))
                g = abs(mc_gluon_1.GetBinContent(j) - mc_gluon_2.GetBinContent(j))
                error_q[j-1] = q
                error_g[j-1] = g

    return error_q,error_g

# do matrix on data for given normalized distribution
def data_matrixmethod(lower_quark_input,lower_gluon_input,higher_quark_input,higher_gluon_input,higher_input,lower_input,fg,cg,fq,cq):
        lower_quark = lower_quark_input.Clone()
        lower_gluon = lower_gluon_input.Clone()
        higher_quark = higher_quark_input.Clone()
        higher_gluon = higher_gluon_input.Clone()
        lower = lower_input.Clone()
        higher = higher_input.Clone()
        if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_quark = higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_quark)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_quark)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_quark)    
        if (doreweight=="Gluon"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_gluon = higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
                                lower_quark.SetBinContent(i,lower_quark.GetBinContent(i)*factor_gluon)
                                lower_gluon.SetBinContent(i,lower_gluon.GetBinContent(i)*factor_gluon)
                                lower.SetBinContent(i,lower.GetBinContent(i)*factor_gluon)                                
        if (lower_quark.Integral() != 0):
            lower_quark.Scale(1./lower_quark.Integral())
        if(lower_gluon.Integral() != 0):
            lower_gluon.Scale(1./lower_gluon.Integral())
        if(higher_quark.Integral() != 0):
            higher_quark.Scale(1./higher_quark.Integral())
        if(higher_gluon.Integral() != 0):
            higher_gluon.Scale(1./higher_gluon.Integral()) 
        if(lower.Integral() != 0):
            lower.Scale(1./lower.Integral()) 
        if(higher.Integral() != 0):
            higher.Scale(1./higher.Integral())         
        #Now, let's solve.
        quark_extracted = higher_quark.Clone()
        gluon_extracted = higher_quark.Clone()
        #if min == 500:
        #    for j in range(1,61):
        #        print("testflag3:",j,higher.GetBinContent(j),lower.GetBinContent(j))

        #Matrix method here
        for i in range(1,higher.GetNbinsX()+1):
                F = higher.GetBinContent(i)
                C = lower.GetBinContent(i)
                if((cg*fq-fg*cq) != 0 ):
                        Q = -(C*fg-F*cg)/(cg*fq-fg*cq)
                        G = (C*fq-F*cq)/(cg*fq-fg*cq)
                        quark_extracted.SetBinContent(i,Q)
                        gluon_extracted.SetBinContent(i,G)
                        #print "   ",i,G,higher_gluon.GetBinContent(i),lower_gluon.GetBinContent(i)
                pass
            
        return(quark_extracted,gluon_extracted)
    
    
# Get the bin(cut) of given working point for the nominal result and the Quark efficiency , Gluon rejection , scale factor.
def wp_sf_nominal(wpoint,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])
    
    mc_bin = np.abs(quark_mc_cumsum -  wpoint*quark_mc.Integral(1,quark_mc.GetNbinsX()+1)).argmin()
    data_bin = np.abs(quark_data_cumsum -  wpoint*quark_data.Integral(1,quark_data.GetNbinsX()+1)).argmin()
    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(mc_bin,data_bin,sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

# Get the Quark efficiency , Gluon rejection , scale factor for systematics.
def wp_sf(mc_bin,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])

    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

#def bootstrap_result(data):
#    w = 58.45/39.91 # prescale factor 
#    n1 = data.Clone()
#    n2 = data.Clone()
#    result = data.Clone()
#    for j in range(1,higher_quark.GetNbinsX()+1):
#            n2.SetBinContent(j,int((data.GetBinContent(j)-data.GetBinError(j)**2)/(w-w**2)))#(events num for prescaled data)
#            n1.SetBinContent(j,int((data.GetBinContent(j)-w*n2.GetBinContent(j))))
#            result.SetBinContent(j,w*np.random.poisson(n2.GetBinContent(j))+np.random.poisson(n1.GetBinContent(j)))
#    return(result)  

def bootstrap_result(data):
    n1 = data.Clone()
    result = data.Clone()
    for j in range(1,higher_quark.GetNbinsX()+1):
            result.SetBinContent(j,np.random.poisson(n1.GetBinContent(j)))
    return(result)  

    
def ReadingDijetDataEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_data = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var)
    lower_data = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var)

    higher_data.SetName(pt+"_LeadingJet_Forward_Data_"+str(index)+var+name)
    lower_data.SetName(pt+"_LeadingJet_Central_Data_"+str(index)+var+name)
    
    higher_data_2 = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var)
    lower_data_2 = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var)
    
    higher_data_2.SetName(pt+"_SubJet_Forward_Data_"+str(index)+var+name)
    lower_data_2.SetName(pt+"_SubJet_Central_Data_"+str(index)+var+name)
    
    higher_data.Add(higher_data_2)
    lower_data.Add(lower_data_2)
    

    higher_data_err = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_err = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_err.SetName(pt+"_LeadingJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_err.SetName(pt+"_LeadingJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_2_err = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_2_err = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_2_err.SetName(pt+"_SubJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_2_err.SetName(pt+"_SubJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_err.Add(higher_data_2_err)
    lower_data_err.Add(lower_data_2_err)
    
    for i in range(1,higher_data.GetNbinsX()+1):
        higher_data.SetBinError(i,np.sqrt(higher_data_err.GetBinContent(i)))
        lower_data.SetBinError(i,np.sqrt(lower_data_err.GetBinContent(i)))
        
    histo_dic["hd"] =  higher_data
    histo_dic["ld"] =  lower_data

    return(histo_dic)        


def ReadingDijetMCEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_quark = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var)


    higher_quark.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+var+name)
    lower_quark.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+var+name)
    higher_gluon.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+var+name)
    

    higher_quark_2 = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark_2 = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon_2 = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon_2 = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark_2 = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark_2 = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark_2 = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark_2 = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var)

    higher_quark_2.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+var+name)
    lower_quark_2.SetName(pt+"_SubJet_Central_Quark_"+str(index)+var+name)
    higher_gluon_2.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon_2.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark_2.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark_2.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark_2.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark_2.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+var+name)

    higher_quark.Add(higher_quark_2)
    higher_gluon.Add(higher_gluon_2)
    higher_b_quark.Add(higher_b_quark_2)
    higher_c_quark.Add(higher_c_quark_2)

    lower_quark.Add(lower_quark_2)
    lower_gluon.Add(lower_gluon_2)
    lower_b_quark.Add(lower_b_quark_2)
    lower_c_quark.Add(lower_c_quark_2)
 
            
    higher_b_quark.Add(higher_c_quark)
    lower_b_quark.Add(lower_c_quark)

    higher_quark_err = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_err = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_err = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_err = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_err = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_err = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_err = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_err = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err")



    
    higher_quark_err.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_err.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_err.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_err.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_err.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_err.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_err.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_err.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_2_err = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_2_err = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_2_err = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_2_err = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_2_err = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_2_err = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_2_err = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_2_err = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err")
    
    higher_quark_2_err.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_2_err.SetName(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_2_err.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_2_err.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_2_err.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_2_err.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_2_err.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_2_err.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_err.Add(higher_quark_2_err)
    higher_gluon_err.Add(higher_gluon_2_err)
    higher_b_quark_err.Add(higher_b_quark_2_err)
    higher_c_quark_err.Add(higher_c_quark_2_err)

    lower_quark_err.Add(lower_quark_2_err)
    lower_gluon_err.Add(lower_gluon_2_err)
    lower_b_quark_err.Add(lower_b_quark_2_err)
    lower_c_quark_err.Add(lower_c_quark_2_err)
 
            
    higher_b_quark_err.Add(higher_c_quark_err)
    lower_b_quark_err.Add(lower_c_quark_err)
    
    for i in range(1,higher_quark.GetNbinsX()+1):
        higher_quark.SetBinError(i,np.sqrt(higher_quark_err.GetBinContent(i)))
        higher_gluon.SetBinError(i,np.sqrt(higher_gluon_err.GetBinContent(i)))
        higher_b_quark.SetBinError(i,np.sqrt(higher_b_quark_err.GetBinContent(i)))
        lower_quark.SetBinError(i,np.sqrt(lower_quark_err.GetBinContent(i)))
        lower_gluon.SetBinError(i,np.sqrt(lower_gluon_err.GetBinContent(i)))
        lower_b_quark.SetBinError(i,np.sqrt(lower_b_quark_err.GetBinContent(i)))

    
    histo_dic["hq"] =  higher_quark
    histo_dic["hg"] =  higher_gluon
    histo_dic["ho"] =  higher_b_quark
    
    histo_dic["hmc"] =  higher_quark.Clone()
    histo_dic["hmc"].SetName(pt+"higher_all_jets"+var+"_"+name)
    histo_dic["hmc"].Add(higher_gluon)
    histo_dic["hmc"].Add(higher_b_quark)
    
    histo_dic["lq"] =  lower_quark
    histo_dic["lg"] =  lower_gluon
    histo_dic["lo"] =  lower_b_quark
    
    histo_dic["lmc"] =  lower_quark.Clone()
    histo_dic["lmc"].SetName(pt+"lower_all_jets"+var+"_"+name)
    histo_dic["lmc"].Add(lower_gluon)
    histo_dic["lmc"].Add(lower_b_quark)
    
    return(histo_dic)

# Get the bin(cut) of given working point for the nominal result and the Quark efficiency , Gluon rejection , scale factor.
def wp_sf_nominal(wpoint,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])
    
    mc_bin = np.abs(quark_mc_cumsum -  wpoint*quark_mc.Integral(1,quark_mc.GetNbinsX()+1)).argmin()
    data_bin = np.abs(quark_data_cumsum -  wpoint*quark_data.Integral(1,quark_data.GetNbinsX()+1)).argmin()
    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(mc_bin,data_bin,sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)

# Get the Quark efficiency , Gluon rejection , scale factor for systematics.
def wp_sf(mc_bin,quark_mc,gluon_mc,quark_data,gluon_data):
    quark_mc_cumsum = np.cumsum(quark_mc)
    gluon_mc_cumsum = np.cumsum(gluon_mc)
    quark_data_cumsum = np.cumsum(quark_data)
    gluon_data_cumsum = np.cumsum(gluon_data)
    #print(quark_mc_cumsum[-1],quark_data_cumsum[-1])

    sf_q = quark_data_cumsum[mc_bin]/quark_mc_cumsum[mc_bin]
    sf_g = (gluon_data_cumsum[-1] - gluon_data_cumsum[mc_bin])/(gluon_mc_cumsum[-1] - gluon_mc_cumsum[mc_bin])
    q_eff_mc = quark_mc_cumsum[mc_bin]/quark_mc_cumsum[-1]
    g_rej_mc = (gluon_mc_cumsum[-1]-gluon_mc_cumsum[mc_bin])/gluon_mc_cumsum[-1]
    q_eff_data = quark_data_cumsum[mc_bin]/quark_data_cumsum[-1]
    g_rej_data = (gluon_data_cumsum[-1]-gluon_data_cumsum[mc_bin])/gluon_data_cumsum[-1]
    #print(gluon_data_cumsum,gluon_mc_cumsum)
    #print(gluon_data_cumsum[data_bin],gluon_mc_cumsum[mc_bin])
    return(sf_q,sf_g,q_eff_mc,g_rej_mc,q_eff_data,g_rej_data)


def bootstrap_result(data):
    n1 = data.Clone()
    result = data.Clone()
    for j in range(1,higher_quark.GetNbinsX()+1):
            result.SetBinContent(j,np.random.poisson(n1.GetBinContent(j)))
    return(result)  




for eta_reweight in reweight_method_array:
    if eta_reweight == 0:
        nevents_eta = []
        for i in range(6):
            nevents_eta.append(np.ones(3))
    

    if eta_reweight == 1:
        nevents_eta = np.load("../mc_eta_events.npy")
        
    if eta_reweight == 2:
        nevents_eta = []
        for i in range(6):
            nevents_eta.append([1,2,3]) 
        
    if eta_reweight == 3:
        nevents_eta = []
        for i in range(6):
            nevents_eta.append([20,10,1])  
    
    
    
    for k in range(5,6):  #for only dijet event, start frTom jet pT>500 GeV

            
            c = TCanvas("c","c",500,500)
            min = bin[k]
            max = bin[k+1]
            print(min)
            
            nominal_jets =  ReadingDijetMCEta(fdijetMC,str(min),var,"0","0")
            
            data_jets = ReadingDijetDataEta(fdijetData,str(min),var,"0","0")

                    
            higher_quark = nominal_jets["hq"]
            lower_quark = nominal_jets["lq"]
            higher_gluon = nominal_jets["hg"]
            lower_gluon = nominal_jets["lg"]
            higher_other = nominal_jets["ho"]
            lower_other = nominal_jets["lo"]        
            higher_mc = nominal_jets["hmc"]
            lower_mc = nominal_jets["lmc"]        

            
            higher_data_TH1I = data_jets["hd"]
            higher_data = higher_quark.Clone(higher_data_TH1I.GetName()+"TH1F")
            for j in range(1,higher_data.GetNbinsX()+1):
                    higher_data.SetBinContent(j,float(higher_data_TH1I.GetBinContent(j)))
                    higher_data.SetBinError(j,float(higher_data_TH1I.GetBinError(j)))
                    
                    
            lower_data_TH1I = data_jets["ld"]
            lower_data = lower_quark.Clone(lower_data_TH1I.GetName()+"TH1F")
            for j in range(1,higher_data.GetNbinsX()+1):
                    lower_data.SetBinContent(j,float(lower_data_TH1I.GetBinContent(j)))
                    lower_data.SetBinError(j,float(lower_data_TH1I.GetBinError(j))) 
                    
            higher_quark.Scale(nevents_eta[k][0])
            higher_gluon.Scale(nevents_eta[k][0])
            higher_other.Scale(nevents_eta[k][0])
            higher_mc.Scale(nevents_eta[k][0])
            lower_quark.Scale(nevents_eta[k][0])
            lower_gluon.Scale(nevents_eta[k][0])
            lower_other.Scale(nevents_eta[k][0])
            lower_mc.Scale(nevents_eta[k][0])
            higher_data.Scale(nevents_eta[k][0])
            lower_data.Scale(nevents_eta[k][0])
            
            print("hq:",higher_quark.Integral()/(higher_quark.Integral()+higher_gluon.Integral()))
            print("lq:",lower_quark.Integral()/(lower_quark.Integral()+lower_gluon.Integral()))

            for e in range(1,len(eta_bin)-1):
        
                    print(var,str(eta_bin[e]),nevents_eta[k][e])
                    
                    nominal_jets_add =  ReadingDijetMCEta(fdijetMC,str(min),var,str(eta_bin[e]),str(eta_bin[e]))
                    # flatten the eta distribution
                        
                    data_jets_add = ReadingDijetDataEta(fdijetData,str(min),var,str(eta_bin[e]),str(eta_bin[e]))
                    
                    higher_data_TH1I_add = data_jets_add["hd"]
                    higher_data_add = higher_quark.Clone(higher_data_TH1I_add.GetName()+"TH1F")
                    for j in range(1,higher_data.GetNbinsX()+1):
                            higher_data_add.SetBinContent(j,float(higher_data_TH1I_add.GetBinContent(j)))
                            higher_data_add.SetBinError(j,float(higher_data_TH1I_add.GetBinError(j)))
                            

                    
                    lower_data_TH1I_add = data_jets_add["ld"]
                    lower_data_add = lower_quark.Clone(lower_data_TH1I_add.GetName()+"TH1F")
                    for j in range(1,higher_data.GetNbinsX()+1):
                            lower_data_add.SetBinContent(j,float(lower_data_TH1I_add.GetBinContent(j)))
                            lower_data_add.SetBinError(j,float(lower_data_TH1I_add.GetBinError(j)))   
                            
                    nominal_jets_add["hq"].Scale(nevents_eta[k][e])
                    nominal_jets_add["hg"].Scale(nevents_eta[k][e])
                    nominal_jets_add["ho"].Scale(nevents_eta[k][e])
                    nominal_jets_add["hmc"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lq"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lg"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lo"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lmc"].Scale(nevents_eta[k][e])
                    higher_data_add.Scale(nevents_eta[k][e])
                    lower_data_add.Scale(nevents_eta[k][e])   
                    
                    higher_quark.Add(nominal_jets_add["hq"])
                    lower_quark.Add(nominal_jets_add["lq"])
                    higher_gluon.Add(nominal_jets_add["hg"])
                    lower_gluon.Add(nominal_jets_add["lg"])
                    higher_other.Add(nominal_jets_add["ho"])
                    lower_other.Add(nominal_jets_add["lo"])        
                    higher_mc.Add(nominal_jets_add["hmc"])
                    lower_mc.Add(nominal_jets_add["lmc"])                    
                    higher_data.Add(higher_data_add)
                    lower_data.Add(lower_data_add) 
                    
                    
                    print("hq:",nominal_jets_add["hq"].Integral()/(nominal_jets_add["hq"].Integral()+nominal_jets_add["hg"].Integral()))
                    print("lq:",nominal_jets_add["lq"].Integral()/(nominal_jets_add["lq"].Integral()+nominal_jets_add["lg"].Integral()))

Welcome to JupyROOT 6.26/04
1500
hq: 0.6853593492056629
lq: 0.6640295765197559
ntrk 0.4 1.0
hq: 0.7380805464581929
lq: 0.7071618456304666
ntrk 0.8 1.0
hq: 0.8346797121229916
lq: 0.7975879175623231
1500
hq: 0.685359349205663
lq: 0.664029576519756
ntrk 0.4 0.9798242349232528
hq: 0.7380805464581928
lq: 0.7071618456304666
ntrk 0.8 3.404129855454994
hq: 0.8346797121229916
lq: 0.7975879175623231
1500
hq: 0.6853593492056629
lq: 0.6640295765197559
ntrk 0.4 2
hq: 0.7380805464581929
lq: 0.7071618456304666
ntrk 0.8 3
hq: 0.8346797121229916
lq: 0.797587917562323
1500
hq: 0.685359349205663
lq: 0.664029576519756
ntrk 0.4 10
hq: 0.7380805464581929
lq: 0.7071618456304665
ntrk 0.8 1
hq: 0.8346797121229916
lq: 

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


0.7975879175623231
